# **Building Dynamic Models Using the Subclassing API**

Both the Sequential API and the Functional API are declarative: you start by declaring which layers you want to use and how they should be connected, and only then can you start feeding the model some data for training or inference.This has many advantages: the model can easily be saved, cloned, shared, its structure can be displayed and analyzed, the framework can infer shapes and check types, so errors can be caught early.

---

But the flip side is just
that: it’s static. Some models involve loops, varying shapes, conditional branching, and other dynamic behaviors. For such cases, or simply if you prefer a more imperative programming style, the Subclassing API is for you.

---

In [4]:
import tensorflow as tf
from tensorflow import keras

In [5]:
class WideAndDeepModel(keras.models.Model):
  def __init__(self, units=30, activation="relu", **kwargs):
    super().__init__(**kwargs) # handles standard args (e.g., name)
    self.hidden1 = keras.layers.Dense(units, activation=activation)
    self.hidden2 = keras.layers.Dense(units, activation=activation)
    self.main_output = keras.layers.Dense(1)
    self.aux_output = keras.layers.Dense(1)
  def call(self, inputs):
    input_A, input_B = inputs
    hidden1 = self.hidden1(input_B)
    hidden2 = self.hidden2(hidden1)
    concat = keras.layers.concatenate([input_A, hidden2])
    main_output = self.main_output(concat)
    aux_output = self.aux_output(hidden2)
    return main_output, aux_output

model = WideAndDeepModel()


In [6]:
class WideAndDeepModel(keras.models.Model):
  def __init__(self, units=30, activation="relu", **kwargs):
    super().__init__(**kwargs) # handles standard args (e.g., name)
    self.hidden1 = keras.layers.Dense(units, activation=activation)
    self.hidden2 = keras.layers.Dense(units, activation=activation)
    self.main_output = keras.layers.Dense(1)
    self.aux_output = keras.layers.Dense(1)
  def call(self, inputs):
    input_A, input_B = inputs # Corrected: 'input' to 'inputs'
    hidden1 = self.hidden1(input_B)
    hidden2 = self.hidden2(hidden1)
    concat = keras.layers.concatenate([input_A, hidden2])
    main_output = self.main_output(concat)
    aux_output = self.aux_output(hidden2)
    return main_output, aux_output

For example, creating an instance of the following WideAndDeepModel class gives us an equivalent model to the one we just built with the Functional API. You can then compile it, evaluate it and use it to make predictions, exactly like we just did.

---

Simply subclass the Model class, create the layers you need in the constructor, and use them to perform the computations you want in the call( ) method.

---

Saving and Restoring Model

In [7]:
model.save("my_keras_model.h5")

In [8]:
with keras.utils.custom_object_scope({'WideAndDeepModel': WideAndDeepModel}):
  model = keras.models.load_model("/content/my_keras_model.h5")
print("Model loaded successfully!")

Model loaded successfully!


Build and Compile Model

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load from Excel (offline)
housing = pd.read_excel("/content/fetch_california_housing.xlsx")

housing

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41,6.984127,1.023810,322,2.555556,37.88,-122.23,4.526
1,8.3014,21,6.238137,0.971880,2401,2.109842,37.86,-122.22,3.585
2,7.2574,52,8.288136,1.073446,496,2.802260,37.85,-122.24,3.521
3,5.6431,52,5.817352,1.073059,558,2.547945,37.85,-122.25,3.413
4,3.8462,52,6.281853,1.081081,565,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25,5.045455,1.133333,845,2.560606,39.48,-121.09,0.781
20636,2.5568,18,6.114035,1.315789,356,3.122807,39.49,-121.21,0.771
20637,1.7000,17,5.205543,1.120092,1007,2.325635,39.43,-121.22,0.923
20638,1.8672,18,5.329513,1.171920,741,2.123209,39.43,-121.32,0.847


In [10]:
y = housing['target']
X = housing.drop(columns='target', axis=1)

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y)

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [12]:
y_train

,target
4813,1.52500
6476,2.07200
18098,3.51200
9657,0.69400
19549,0.79500
...,...
15681,5.00001
6901,2.03300
8462,1.81800
16662,2.47000


In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

y_train_scaled = scaler.fit_transform(
    y_train.to_numpy().reshape(-1, 1)
)


In [14]:
y_valid_scaled = scaler.fit_transform(
    y_valid.to_numpy().reshape(-1, 1)
)

In [15]:
import numpy as np

np.isnan(X_train_scaled).any()
np.isnan(y_train).any()
np.isinf(X_train_scaled).any()


np.False_

Build and Compile Model

In [16]:
model = keras.models.Sequential([
    keras.layers.Input(shape=X_train_scaled.shape[1:]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer="sgd"
)

In [17]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5")
history = model.fit(X_train_scaled, y_train_scaled, epochs=10, callbacks=[checkpoint_cb])

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7536

363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.7531
Epoch 2/10
339/363 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3926

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4014
Epoch 3/10
344/363 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3749

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3745
Epoch 4/10
348/363 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3470

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3469
Epoch 5/10
343/363 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3297

...

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3049
Epoch 7/10
347/363 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2923

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2935
Epoch 8/10
353/363 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2892

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2898
Epoch 9/10
350/363 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3143

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3139
Epoch 10/10
349/363 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2960

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2960


Moreover, if you use a validation set during training, you can set
save_best_only=True when creating the ModelCheckpoint. In this case, it will only save your model when its performance on the validation set is the best so far.

---

This way, you do not need to worry about training for too long and overfitting the training set: simply restore the last model saved after training, and this will be the best model on the validation set. This is a simple way to implement early stopping.

---

Saved the best model.

In [18]:
# checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
# history = model.fit(X_train_scaled, y_train_scaled, epochs=10, validation_data=(X_valid_scaled, y_valid_scaled), callbacks=[checkpoint_cb])


# model = keras.models.load_model("my_keras_model.h5") # rollback to best model

---

Another way to implement early stopping is to simply use the EarlyStopping call‐
back. It will interrupt training when it measures no progress on the validation set for a number of epochs (defined by the patience argument), and it will optionally roll back to the best model

---

# **Early Stopping**

In [19]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "my_keras_model.keras",
    save_best_only=True
)

early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True
)


In [20]:
history = model.fit(
    X_train_scaled,
    y_train_scaled,
    epochs=20,
    validation_data=(X_valid_scaled, y_valid_scaled),
    callbacks=[checkpoint_cb, early_stopping_cb]
)

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.2861 - val_loss: 0.2951
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2895 - val_loss: 0.2927
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2949 - val_loss: 0.2904
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2716 - val_loss: 0.2821
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2795 - val_loss: 0.2829
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2729 - val_loss: 0.2790
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2902 - val_loss: 0.2761
Epoch 8/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2714 - val_loss: 0.2756
Epoch 9/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2748 - val_loss: 0.2714
Epoch 10/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2661 - val_loss: 0.2682
Epoch 11/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2578 - val_loss: 0.2676
Epoch 12/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

There are many other callbacks available in the keras.callbacks package. See

---

https://keras.io/api/callbacks/

# **Visualization Using TensorBoard**

TensorBoard is a great interactive visualization tool that you can use to view the learning curves during training, compare learning curves between multiple runs, visualize the computation graph, analyze training statistics, view images generated by your model, visualize complex multidimensional data projected down to 3D and automatically clustered for you, and more!

In [21]:
import os

root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
  import time
  run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
  return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir() # e.g., './my_logs/run_2019_01_16-11_28_43'


# Next, the good news is that Keras provides a nice TensorBoard callback:


# Build and Compile Model
model = keras.models.Sequential([
    keras.layers.Input(shape=X_train_scaled.shape[1:]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer="sgd"
)

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train_scaled, y_train_scaled, epochs=30,
 validation_data=(X_valid_scaled, y_valid_scaled),
 callbacks=[tensorboard_cb])

Epoch 1/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.7725 - val_loss: 0.4212
Epoch 2/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4188 - val_loss: 0.3593
Epoch 3/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3603 - val_loss: 0.3370
Epoch 4/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3384 - val_loss: 0.3243
Epoch 5/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3345 - val_loss: 0.3171
Epoch 6/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3256 - val_loss: 0.3139
Epoch 7/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3037 - val_loss: 0.3081
Epoch 8/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3005 - val_loss: 0.3015
Epoch 9/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.2930 - val_loss: 0.2981
Epoch 10/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3080 - val_loss: 0.2902
Epoch 11/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2919 - val_loss: 0.2882
Epoch 12/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step